In [1]:
import numpy as np
import pandas as pd

from sqlalchemy import select, create_engine, MetaData, func, intersect


engine = create_engine('sqlite://', echo=False)
connect = engine.connect()


# Connect the required databases
chicago_crime = pd.read_csv('https://data.cityofchicago.org/resource/ijzp-q8t2.csv')
cenus_data = pd.read_csv('https://data.cityofchicago.org/resource/kn9c-c2s2.csv')
schools = pd.read_csv('https://data.cityofchicago.org/resource/9xs2-f89t.csv')

chicago_crime.to_sql('chicago_crime', con=engine)
cenus_data.to_sql('cenus_data', con=engine)
schools.to_sql('schools', con=engine)


# Заполнить существующую базу данных с помощью отражения в объекты sqlalchemy
db = MetaData()
db.reflect(bind=engine)

# Посмотрим какие таблицы есть в db
name_db = db.sorted_tables
for t in name_db:
    print(t.name)

# превращаем DataFrame в Table
chicago_crime = db.tables['chicago_crime']
cenus_data = db.tables['cenus_data']
schools = db.tables['schools']

cenus_data
chicago_crime
schools


In [2]:
# Задание 1. Найдите общее количество преступлений, записанных в таблице Crime.

# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT count(*) AS count
# FROM chicago_crime;
# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


# Выведем количество строк в таблице chicago_crime
# Результирующая переменная является эквивалентом cursor в DB API.
count = connect.execute(select(func.count().label('count')).select_from(chicago_crime))
print('Total number of crimes: ', count.fetchone())

Total number of crimes:  (1000,)


In [3]:
# Задание 2. Получите первые 10 строк из таблицы CRIME.

# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT *
# FROM chicago_crime
# LIMIT 10;
# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


head = connect.execute(select(chicago_crime).limit(10))
row = head.fetchone()
while row is not None:
    print(row)
    row = head.fetchone()

(0, 12746194, 'JF299386', '2022-06-29T23:57:00.000', '045XX S HERMITAGE AVE', '0554', 'ASSAULT', 'AGGRAVATED POLICE OFFICER - HANDS, FISTS, FEET, NO INJURY', 'STREET', True, False, 924, 9, 15, 61, '08A', 1165438.0, 1874640.0, 2022, '2022-07-06T16:50:14.000', 41.811605332, -87.668700581, '\n,  \n(41.811605332, -87.668700581)')
(1, 12746184, 'JF299379', '2022-06-29T23:55:00.000', '009XX W LAKESIDE PL', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'VEHICLE NON-COMMERCIAL', False, False, 1914, 19, 46, 3, '14', 1169180.0, 1931740.0, 2022, '2022-07-06T16:50:14.000', 41.968211376, -87.653315307, '\n,  \n(41.968211376, -87.653315307)')
(2, 12746225, 'JF299368', '2022-06-29T23:53:00.000', '051XX W MADISON ST', '0320', 'ROBBERY', 'STRONG ARM - NO WEAPON', 'SIDEWALK', False, False, 1533, 15, 28, 25, '03', 1142285.0, 1899542.0, 2022, '2022-07-06T16:50:14.000', 41.880399914, -87.753008553, '\n,  \n(41.880399914, -87.753008553)')
(3, 12746245, 'JF299446', '2022-06-29T23:51:00.000', '087XX S SOUTH CHICA

In [4]:
# Задание 3. По скольким преступлениям был произведен арест?

# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT count(*) AS arrest
# FROM chicago_crime
# WHERE chicago_crime.arrest = True
# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


arrest = select(func.count().label('arrest')).select_from(chicago_crime).where(chicago_crime.c.arrest == True)
for row in connect.execute(arrest):
    print(row)

(134,)


In [5]:
# Задание 4. Какие уникальные виды преступлений совершались на автозаправочных станциях (GAS STATION)?

# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT DISTINCT chicago_crime.primary_type
# FROM chicago_crime
# WHERE chicago_crime.location_description = 'GAS STATION'
# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


unique_primary_type = select(chicago_crime.c.primary_type).select_from(chicago_crime).where(chicago_crime.c.location_description == 'GAS STATION').distinct()
for row in connect.execute(unique_primary_type):
    print(row)


('CRIMINAL TRESPASS',)
('THEFT',)
('OTHER OFFENSE',)
('WEAPONS VIOLATION',)
('BATTERY',)
('ROBBERY',)
('ASSAULT',)
('NARCOTICS',)


In [6]:
# Задание 5. Получите из таблицы данных переписи CENUS_DATA все районы (community areas), названия которых начинаются с буквы «B» и в которых совершались преступления.

# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT cenus_data.community_area_name
# FROM cenus_data
# WHERE
# cenus_data.community_area_name LIKE 'B%' AND
# cenus_data.ca IN (SELECT DISTINCT chicago_crime.community_area FROM chicago_crime)
# +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


subq = select(chicago_crime.c.community_area).select_from(chicago_crime).distinct()
criminal_areas = select(cenus_data.c.community_area_name).select_from(cenus_data).where(
    cenus_data.c.community_area_name.like('B%'), cenus_data.c.ca.in_(subq)
)
for row in connect.execute(criminal_areas):
    print(row)

('Belmont Cragin',)
('Burnside',)
('Bridgeport',)
('Beverly',)


In [7]:
# Задание 6. Получите список школ в округах с 10 по 15, имеющих сертификат здоровья (healthy school certified)

#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT schools.name_of_school
# FROM schools
# WHERE schools.police_district >= 10 AND schools.police_district <= 15 AND schools.healthy_schools_certified_ = 'Yes'
#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


healthy_school = select(schools.c.name_of_school).select_from(schools).where(
    schools.c.police_district >= 10, schools.c.police_district <= 15, schools.c.healthy_schools_certified_ == 'Yes'
)
for row in connect.execute(healthy_school):
    print(row)

('Ana Roque de Duprey Elementary School',)
('Cyrus H McCormick Elementary School',)
('Eli Whitney Elementary School',)
('Francisco I Madero Middle School',)
('Irma C Ruiz Elementary School',)
('John A Walsh Elementary School',)
('Jose De Diego Elementary Community Academy',)
('Manuel Perez Elementary School',)


In [8]:
# Задание 7. Вычислите средний рейтинг безопасности (Safety Score) всех школ.

#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT avg(schools.safety_score) AS avg
# FROM schools
#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


avg_safety_score = select(func.avg(schools.c.safety_score).label('avg'))
for row in connect.execute(avg_safety_score):
    print(row)

(49.50487329434698,)


In [9]:
# Задание 8. Получите список из 5 районов с наибольшим средним количеством учеников, поступивших в колледж (College Enrollments).

#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT s.college_enrollment_number_of_students_, s.community_area_name
# FROM
# (SELECT * FROM schools WHERE schools.name_of_school LIKE '%College%') AS s
# ORDER BY s.college_enrollment_number_of_students_ DESC
#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


subq = select(schools).where(schools.c.name_of_school.like('%College%')).subquery()
smart_area = select(subq.c.college_enrollment_number_of_students_, subq.c.community_area_name).select_from(subq).order_by(subq.c.college_enrollment_number_of_students_.desc()).limit(5)
for row in connect.execute(smart_area):
    print(row)

(1053, 'NORTH PARK')
(967, 'WEST ELSDON')
(915, 'KENWOOD')
(881, 'NEAR NORTH SIDE')
(871, 'LOOP')


In [10]:
# Задание 9. Найдите район с самым низким рейтингом безопасности (Safety Score), используя подзапрос.

#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT subq.community_area_name
# FROM (SELECT schools.community_area_name AS community_area_name, min(schools.safety_score) FROM schools) AS subq
#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


subq = select(schools.c.community_area_name, func.min(schools.c.safety_score)).select_from(schools).subquery()
low_safety_score = select(subq.c.community_area_name).select_from(subq)
for row in connect.execute(low_safety_score):
    print(row)

('WASHINGTON PARK',)


In [11]:
# Задание 10. Получите информацию о доходе на душу населения в районе, где рейтинг безопасности школ равен 1.

#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# SELECT cenus_data.community_area_name 
# FROM cenus_data 
# WHERE lower(cenus_data.community_area_name) IN (
#     SELECT lower(schools.community_area_name) AS lower_1 
#     FROM schools 
#     WHERE schools.safety_score = 1
# )
#+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+

schools_area = select(func.lower(schools.c.community_area_name)).select_from(schools).where(schools.c.safety_score == 1)
per_capita_income = select(cenus_data.c.community_area_name).select_from(cenus_data).where(
    func.lower(cenus_data.c.community_area_name).in_(schools_area)
)
for row in connect.execute(per_capita_income):
    print(row)

('Washington Park',)
